In [19]:
from statsmodels.regression.quantile_regression import QuantReg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_pinball_loss
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

## Read Train and Test Data 

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

## Check for Missing values and replace with the most frequent value

In [3]:
train.isnull().sum()

id                 0
wi                 0
year               0
month              0
age                0
education          0
familysize         0
urban              0
race               0
region          6994
state          95742
marital            0
occupation    233483
income             0
expense            0
dtype: int64

In [4]:
train = train.fillna(train.mode().iloc[0].astype('int64'))
train.isnull().sum()
train.dtypes

id              int64
wi            float64
year            int64
month           int64
age             int64
education       int64
familysize      int64
urban           int64
race            int64
region        float64
state         float64
marital         int64
occupation    float64
income          int64
expense         int64
dtype: object

In [5]:
col_types={'region':'int64','state':'int64','occupation':'int64'}
train = train.astype(col_types)

In [6]:
test.isnull().sum()

id                0
year              0
month             0
age               0
education         0
familysize        0
urban             0
race              0
region         1826
state         24016
marital           0
occupation    58171
income            0
dtype: int64

In [7]:
test = test.fillna(test.mode().iloc[0].astype('int64'))
test.isnull().sum()
test.dtypes

id              int64
year            int64
month           int64
age             int64
education       int64
familysize      int64
urban           int64
race            int64
region        float64
state         float64
marital         int64
occupation    float64
income          int64
dtype: object

In [8]:
col_types={'region':'int64','state':'int64','occupation':'int64'}
test = test.astype(col_types)

## Identify categorical, ordinal and numeric columns

In [9]:
columns = train.columns
columns  = columns.to_list()
categorical_columns=['urban','race','region','state','marital','occupation']
ordinal_columns=['education']
numeric_columns=['age','familysize','income']
weight_column=['wi']
target=['expense']
id_column=['id']
columns

['id',
 'wi',
 'year',
 'month',
 'age',
 'education',
 'familysize',
 'urban',
 'race',
 'region',
 'state',
 'marital',
 'occupation',
 'income',
 'expense']

## Convert types of various columns in train and test data

In [10]:
#Convert cateogrical columns to 'category' type
cat_types={c:'category' for c in categorical_columns}
train = train.astype(cat_types)
#Convert ordinal column to category type
train['education'] = train['education'].astype('category')
#Get different categories for ordinal column
train_ord_categories = train.education.unique().tolist()
train_ord_categories.sort()
train['education'] = train['education'].cat.set_categories(train_ord_categories,ordered=True)

In [11]:
#Convert cateogrical columns to 'category' type
cat_types={c:'category' for c in categorical_columns}
test = test.astype(cat_types)
#Convert ordinal column to category type
test['education'] = test['education'].astype('category')
#Get different categories for ordinal column
test_ord_categories = test.education.unique().tolist()
test_ord_categories.sort()
test['education'] = test['education'].cat.set_categories(test_ord_categories,ordered=True)
test.dtypes

id               int64
year             int64
month            int64
age              int64
education     category
familysize       int64
urban         category
race          category
region        category
state         category
marital       category
occupation    category
income           int64
dtype: object

## Create dummy categorical columns for all categorical variables in train

In [12]:
train_cat_dummies = pd.get_dummies(train[categorical_columns])
new_cat_columns=train_cat_dummies.columns.to_list()
train = pd.concat([train,train_cat_dummies],axis=1)
train.drop(categorical_columns,inplace=True,axis=1)
train.drop(['year','month'],inplace=True,axis=1)
train['education']=train['education'].cat.codes
train

,id,wi,age,education,familysize,income,expense,urban_1,urban_2,race_1,...,occupation_9,occupation_10,occupation_11,occupation_12,occupation_13,occupation_14,occupation_15,occupation_16,occupation_17,occupation_18
0,1,2831.0,50,3,3,13141,2398,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1941.0,23,4,1,0,575,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,3,1369.0,56,6,4,459,1592,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,4,816.0,59,8,2,17474,3443,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,3064.0,51,6,2,39395,1484,1,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
741869,741870,1515.0,66,7,2,7780,9670,1,0,1,...,0,0,0,0,0,0,0,0,0,0
741870,741871,1544.0,25,5,1,12528,1146,1,0,1,...,0,0,0,0,0,0,0,0,0,0
741871,741872,2702.0,56,6,2,37435,3077,1,0,0,...,0,0,0,0,0,0,0,0,0,0
741872,741873,1310.0,48,6,5,54714,6566,1,0,1,...,0,0,0,0,0,0,0,0,0,0


## Create dummy categorical columns for all categorical variables in test

In [13]:
test_cat_dummies = pd.get_dummies(test[categorical_columns])
test_new_cat_columns=test_cat_dummies.columns.to_list()
test = pd.concat([test,test_cat_dummies],axis=1)
test.drop(categorical_columns,inplace=True,axis=1)
test.drop(['year','month'],inplace=True,axis=1)
test['education']=test['education'].cat.codes
test

,id,age,education,familysize,income,urban_1,urban_2,race_1,race_2,race_3,...,occupation_9,occupation_10,occupation_11,occupation_12,occupation_13,occupation_14,occupation_15,occupation_16,occupation_17,occupation_18
0,741875,49,4,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,741876,29,3,4,11628,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,741877,22,4,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,741878,78,3,2,13413,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,741879,25,6,2,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185464,927339,51,4,2,20957,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
185465,927340,70,2,1,3843,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
185466,927341,58,6,1,15598,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
185467,927342,35,4,5,10138,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


## Standardize numerical columns in train

In [14]:
for c in numeric_columns:
    train[c] = (train[c]-train[c].mean())/train[c].std()
train

,id,wi,age,education,familysize,income,expense,urban_1,urban_2,race_1,...,occupation_9,occupation_10,occupation_11,occupation_12,occupation_13,occupation_14,occupation_15,occupation_16,occupation_17,occupation_18
0,1,2831.0,0.016812,3,0.321927,-0.287200,2398,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1941.0,-1.538223,4,-1.010164,-0.995756,575,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,3,1369.0,0.362376,6,0.987973,-0.971007,1592,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,4,816.0,0.535157,8,-0.344119,-0.053567,3443,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,3064.0,0.074406,6,-0.344119,1.128403,1484,1,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
741869,741870,1515.0,0.938314,7,-0.344119,-0.576263,9670,1,0,1,...,0,0,0,0,0,0,0,0,0,0
741870,741871,1544.0,-1.423035,5,-1.010164,-0.320253,1146,1,0,1,...,0,0,0,0,0,0,0,0,0,0
741871,741872,2702.0,0.362376,6,-0.344119,1.022721,3077,1,0,0,...,0,0,0,0,0,0,0,0,0,0
741872,741873,1310.0,-0.098375,6,1.654018,1.954396,6566,1,0,1,...,0,0,0,0,0,0,0,0,0,0


## Standardize numerical columns in test

In [15]:
for c in numeric_columns:
    test[c] = (test[c]-test[c].mean())/test[c].std()
test

,id,age,education,familysize,income,urban_1,urban_2,race_1,race_2,race_3,...,occupation_9,occupation_10,occupation_11,occupation_12,occupation_13,occupation_14,occupation_15,occupation_16,occupation_17,occupation_18
0,741875,-0.035942,4,-1.011089,-0.994219,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,741876,-1.183839,3,0.992162,-0.368123,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,741877,-1.585604,4,-1.011089,-0.994219,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,741878,1.628510,3,-0.343339,-0.272012,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,741879,-1.413419,6,-0.343339,-0.994219,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185464,927339,0.078848,4,-0.343339,0.134186,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
185465,927340,1.169351,2,-1.011089,-0.787298,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
185466,927341,0.480612,6,-1.011089,-0.154363,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
185467,927342,-0.839470,4,1.659912,-0.448351,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


## Create Train and Validation

In [16]:
columns = train.columns.to_list()
from sklearn.model_selection import train_test_split
a_train,a_validation = train_test_split(train,test_size=0.2)
train = pd.DataFrame(a_train,columns=columns)
validation = pd.DataFrame(a_validation,columns=columns)

## Create X_train, y_train, X_val, y_val

In [24]:
X_train = train[new_cat_columns+ordinal_columns+numeric_columns]
y_train = train[target]
sample_weights_train=train[weight_column].to_numpy()
sample_weights_train=sample_weights_train.reshape(-1)
X_val = validation[new_cat_columns+ordinal_columns+numeric_columns]
y_val = validation[target]
sample_weights_validation=validation[weight_column].to_numpy()
sample_weights_validation = sample_weights_validation.reshape(-1)

## Create X_test for final predictions

In [25]:
X_test = test[test_new_cat_columns+ordinal_columns+numeric_columns]
input_dims_test=X_test.shape[1]
print(input_dims_test)

84


## add constant term to X_train,X_val,X_test

In [26]:
X_train = sm.add_constant(X_train)
print(X_train)
X_val = sm.add_constant(X_val)
print(X_val)
X_test = sm.add_constant(X_test)
print(X_test)

        const  urban_1  urban_2  race_1  race_2  race_3  race_4  race_5  \
148357    1.0        1        0       1       0       0       0       0   
444136    1.0        1        0       1       0       0       0       0   
720759    1.0        1        0       0       0       0       0       0   
401073    1.0        1        0       1       0       0       0       0   
199128    1.0        1        0       1       0       0       0       0   
...       ...      ...      ...     ...     ...     ...     ...     ...   
125714    1.0        1        0       1       0       0       0       0   
621810    1.0        1        0       1       0       0       0       0   
633005    1.0        1        0       1       0       0       0       0   
620498    1.0        1        0       1       0       0       0       0   
644730    1.0        1        0       1       0       0       0       0   

        race_6  region_1  ...  occupation_13  occupation_14  occupation_15  \
148357       0       

## Create models for each quantile

In [29]:
quantiles = [0.005,0.025,0.165,0.25,0.5,0.75,0.835,0.975,0.995]
models = {}
for index,q in enumerate(quantiles):
    models["q"+str(index+1)] = QuantReg(y_train,X_train)

## fit model for each quantile

In [30]:
results={}
for i,q in enumerate(models.keys()):
    print(f"fitting model for {q}")
    results[q]=models[q].fit(q=quantiles[i])
    print(results[q].summary())

fitting model for q1


C:\Users\pmoha\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1934: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(eigvals[0]/eigvals[-1])


                         QuantReg Regression Results                          
Dep. Variable:                expense   Pseudo R-squared:            6.465e-05
Model:                       QuantReg   Bandwidth:                       120.4
Method:                 Least Squares   Sparsity:                        4679.
Date:                Thu, 08 Jun 2023   No. Observations:               593499
Time:                        19:48:03   Df Residuals:                   593420
                                        Df Model:                           78
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           -11.2203      2.792     -4.019      0.000     -16.692      -5.748
urban_1          -5.6101      2.347     -2.390      0.017     -10.211      -1.009
urban_2          -5.6101      3.170     -1.770      0.077     -11.824       0.603
race_1           -1.8700      4.015  

C:\Users\pmoha\anaconda3\lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
C:\Users\pmoha\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1934: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(eigvals[0]/eigvals[-1])


                         QuantReg Regression Results                          
Dep. Variable:                expense   Pseudo R-squared:             0.001111
Model:                       QuantReg   Bandwidth:                       89.71
Method:                 Least Squares   Sparsity:                        3802.
Date:                Thu, 08 Jun 2023   No. Observations:               593499
Time:                        19:53:07   Df Residuals:                   593420
                                        Df Model:                           78
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            28.4614      2.589     10.991      0.000      23.386      33.537
urban_1          14.2307      2.069      6.878      0.000      10.176      18.286
urban_2          14.2307      2.781      5.118      0.000       8.781      19.681
race_1            4.7436      3.591  

...

C:\Users\pmoha\anaconda3\lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
C:\Users\pmoha\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1934: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(eigvals[0]/eigvals[-1])


                         QuantReg Regression Results                          
Dep. Variable:                expense   Pseudo R-squared:               0.2187
Model:                       QuantReg   Bandwidth:                       60.52
Method:                 Least Squares   Sparsity:                        7195.
Date:                Thu, 08 Jun 2023   No. Observations:               593499
Time:                        20:14:49   Df Residuals:                   593420
                                        Df Model:                           78
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          1198.6149     10.292    116.458      0.000    1178.442    1218.787
urban_1         621.0585      8.241     75.362      0.000     604.906     637.211
urban_2         577.5564     11.197     51.580      0.000     555.610     599.503
race_1          326.7019     13.859  

C:\Users\pmoha\anaconda3\lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
C:\Users\pmoha\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1934: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(eigvals[0]/eigvals[-1])


                         QuantReg Regression Results                          
Dep. Variable:                expense   Pseudo R-squared:               0.2043
Model:                       QuantReg   Bandwidth:                       186.8
Method:                 Least Squares   Sparsity:                    9.970e+04
Date:                Thu, 08 Jun 2023   No. Observations:               593499
Time:                        20:19:56   Df Residuals:                   593420
                                        Df Model:                           78
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          2640.6033     61.167     43.170      0.000    2520.717    2760.490
urban_1        1315.0247     48.977     26.850      0.000    1219.032    1411.018
urban_2        1325.5785     66.386     19.968      0.000    1195.465    1455.693
race_1          887.7659     80.172  

C:\Users\pmoha\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1934: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(eigvals[0]/eigvals[-1])


## Predict on entire training set and calculate mean_pinball_loss_train for all quantiles

In [35]:
mpl_train={}
for i,q in enumerate(models.keys()):
    y_pred=models[q].predict(results[q].params,X_train)
    mpl_train[q]=mean_pinball_loss(y_train.values.ravel(),y_pred,sample_weight=sample_weights_train,alpha=quantiles[i])
mpl_train

{'q1': 20.893509565491406,
 'q2': 77.98099633714666,
 'q3': 355.0541424715648,
 'q4': 475.4611229449974,
 'q5': 707.5757445423578,
 'q6': 740.956586076636,
 'q7': 681.3720826479515,
 'q8': 340.343942807233,
 'q9': 149.41295177290075}

## Predict on validation set and calculate mean_pinball_loss for all quantiles

In [36]:
mpl_val={}
for i,q in enumerate(models.keys()):
    y_pred = models[q].predict(results[q].params,X_val)
    mpl_val[q]=mean_pinball_loss(y_val.values.ravel(),y_pred,sample_weight=sample_weights_validation,alpha=quantiles[i])
mpl_val

{'q1': 20.81705721061542,
 'q2': 78.0797100273436,
 'q3': 355.6157963385023,
 'q4': 476.4900107764522,
 'q5': 708.939691182505,
 'q6': 742.1761536311748,
 'q7': 682.1758487500197,
 'q8': 340.28182587108313,
 'q9': 150.30438422891382}

## write mean pinball losses to a file

In [41]:
mpls={}
mpls['Data'] = ['mpl_train','mpl_val']
for q in mpl_train.keys():
    mpls[q]=[mpl_train[q],mpl_val[q]]
df = pd.DataFrame(mpls)
df.to_csv("QuantReg_MeanPinballLoss.csv",index=False)

## Predict on test set and create output data frame

In [37]:
test_df = test[id_column]
for q in models.keys():
    test_df[q] = models[q].predict(results[q].params,X_test)

C:\Users\pmoha\AppData\Local\Temp\ipykernel_13776\3625866153.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[q] = models[q].predict(results[q].params,X_test)
C:\Users\pmoha\AppData\Local\Temp\ipykernel_13776\3625866153.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[q] = models[q].predict(results[q].params,X_test)
C:\Users\pmoha\AppData\Local\Temp\ipykernel_13776\3625866153.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [38]:
test_df

,id,q1,q2,q3,q4,q5,q6,q7,q8,q9
0,741875,2.625284e-07,1.335710e-06,425.750467,537.623609,794.949463,1191.053305,1378.947152,2295.356291,5807.003469
1,741876,1.514087e-07,8.737303e-07,1415.464612,1703.422513,2406.402876,3354.311509,4029.781911,8537.502435,16650.559783
2,741877,1.887519e-07,9.372171e-07,453.669447,587.751701,862.119392,1223.645269,1479.823588,3676.267796,9543.561733
3,741878,2.285733e-07,1.510000e+02,875.965464,1076.339992,1635.075079,2487.974428,3009.968008,7049.815666,16005.727508
4,741879,2.326167e-07,8.549312e-07,728.736587,907.642629,1298.113729,1797.541851,2098.594787,4198.309816,8248.462514
...,...,...,...,...,...,...,...,...,...,...
185464,927339,1.017788e-07,9.338128e-07,1458.698941,1746.577024,2568.482532,3696.389567,4387.404839,10256.172128,23627.670392
185465,927340,3.161272e-07,1.510000e+02,507.201904,616.156098,898.721887,1373.792307,1664.355275,3394.964589,7833.424167
185466,927341,9.545203e-08,9.668035e-07,1183.539871,1469.515760,2198.549230,3261.360500,3956.618622,9267.020527,20249.523966
185467,927342,1.799198e-07,5.594043e-07,1378.417483,1631.776092,2258.041714,3042.298962,3488.234037,6678.035692,13116.177588


## write output to test_quantiles_LinearModelQuantReg.csv

In [39]:
test_df.to_csv('test_quantiles_LinearModelQuantReg.csv',index=False)